Maxime Marchand
# Astrophysics and Data Science : Project 5
## Neural network from scrach : Part 3

In this part, we train our neural network on the MNIST database, representing handwritten digits between 0 and 9. The dataset is made of 60'000 training digits, and 10'000 validation digits.

In general, the images are first convolved and then reduced in order to decrease the number of input dimention, this code is just entering each pixels in the input layer, which is clearly not the most optimal implementation. But we still have ~87 % accuracy.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from _05_functions import *   # Functions presented in Part 1

The input files are really big, so after having applied the same steps presented in Part 1 on the data (linear normalization, separating pixels and classes, creating targets, etc...), the variables have been saved with the pickle package in order to make the code run faster. The commented cell below is what has been made to prepare the data.

In [ ]:
# File names --------------------------
#trainFileName = 'Data/mnist_train.csv'
#testFileName  = 'Data/mnist_test.csv'

# Collecting data ---------------------------------------------------------
#train_data    = np.loadtxt(trainFileName, delimiter=',', comments='label')
#test_data     = np.loadtxt(testFileName, delimiter=',', comments='label')

# Creating targets -----------------------------------------------
#train_targets = np.zeros(shape=(len(training_set['vectors']),10))
#test_targets  = np.zeros(shape=(len(testing_set['vectors']), 10))
#
#for i in range(len(train_targets)):
#    train_targets[i, int(train_data[i, 0])] = np.ones(1)
#
#for i in range(len(test_targets)):
#    test_targets[i, int(test_data[i, 0])] = np.ones(1)
#
#training_set = {
#    'vectors'  : train_data[:, 1:],
#    'classes'  : train_data[:, 0],
#    'targets'  : train_targets
#}
#
#testing_set = {
#    'vectors'  : test_data[:, 1:],
#    'classes'  : test_data[:, 0],
#    'targets'  : test_targets    
#}

# Normalize the data -------------------------
#for i in range(len(training_set['vectors'])):
#    minim = np.min(training_set['vectors'][i])
#    maxim = np.max(training_set['vectors'][i])
#    training_set['vectors'][i] = (2 * (training_set['vectors'][i] - minim) / (maxim - minim)) - 1

#Saving variables with pickle ------------------------
#with open('./Data/mnist_train.pkl', 'wb') as outFile:
#    pickle.dump(training_set, outFile)
#with open('./Data/mnist_test.pkl', 'wb') as outFile:
#    pickle.dump(testing_set, outFile)

In [ ]:
#Loading variables with pickle
with open('./Data/mnist_train.pkl', 'rb') as inFile:
    training_set = pickle.load(inFile)
with open('./Data/mnist_test.pkl', 'rb') as inFile:
    testing_set = pickle.load(inFile)
    
class_dim = len(np.unique(training_set['classes']))  # Number of possible outputs (3 in the case of wheat database)
data_dim  = len(training_set['vectors'][0])          # Dimention of the data (7 in the case of wheat database)
transfert_mode = 'sig'                               # Choose between 'sig', 'ReLU' and 'ReLU_dying'

Let have a look at some of the digits

In [ ]:
numrows, numcols = 5, 10
fig, ax = plt.subplots(nrows=numrows, ncols=numcols, figsize=(16, 8))

idx=0 # Counter
for i in range(numrows):
    for j in range(numcols):
        ax[i, j].xaxis.set_visible(False)
        ax[i, j].yaxis.set_visible(False)
        ax[i, j].imshow(training_set['vectors'][numrows*i+j].reshape((28, 28)), cmap='gray')
        idx += 1

In [ ]:
#class_dim = len(np.unique(training_set['classes']))  # Number of possible outputs (3 in the case of wheat database)
#data_dim  = len(training_set['vectors'][0])          # Dimention of the data (7 in the case of wheat database)

# Architecture of the network
#num_neurons_input  = data_dim
#num_hidden_layers  = 1
#num_neurons_hidden = 20
#num_neurons_output = class_dim

# Training parameters
#num_epochs     = 1
#learning_rate  = 0.05
#transfert_mode = 'sig'   # Choose between 'sig', 'ReLU' and 'ReLU_dying'

#network = generate_neural_network(num_neurons_input, num_hidden_layers, num_neurons_hidden, num_neurons_output)
#network, vec_cost = train_network(network, num_epochs, learning_rate, training_set, transfert_mode)

The training of the network takes quite a long time, so I trained it, and saved the state of the network with the pickle module.

In [ ]:
with open('./Data/network.pkl', 'rb') as inFile:
    network = pickle.load(inFile)

In [ ]:
def class_from_output(output):
    """
    Converts target vector to class
    
    PARAMETERS
        output : np.array
        
    RETURNS
        0 if output == [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        1 if output == [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
        2 if output == [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
        ...
        9 if output == [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
    """
    return float(np.where(output == np.max(output))[0])

In [ ]:
# We test the network with the validation data set
predicted_values = np.ndarray(len(testing_set['vectors']))

for i in range(len(testing_set['vectors'])):
    entry      = testing_set['vectors'][i]                              # Data to put in network
    network    = forward_propagation(network, entry, transfert_mode='sig') # Forward propagation 
    net_output = network[-1]['O']                                          # Predicted value from network
    predicted_values[i] = class_from_output(net_output)                    # Storing the predicted class

CM = np.zeros(shape=(class_dim, class_dim)) # Confusion Matrix

for c in range(0, 10):     # c - class
    for p in range(0, 10): # p - prediction
        sel = (predicted_values == p) * (testing_set['classes'] == c)   # The * operator acts as the logic gate AND
        CM[c, p] = len(sel[sel])

print("Confusion matrix \n")
print("{:<10} : {:>6} {:>6} {:>6} {:>6} {:>6} {:>6} {:>6} {:>6} {:>6} {:>6}".format("Predicted", *np.arange(10)))
print("Classes")
for i in range(len(CM)):
    [print("{:^12} {:>6} {:>6} {:>6} {:>6} {:>6} {:>6} {:>6} {:>6} {:>6} {:>6}".format(i, *CM[i]))];
    
print("\nAccuracy : {:.2f}%".format(100*np.trace(CM)/np.sum(CM)))

In [ ]:
# Change idx to test some digits
idx = 8
entry      = testing_set['vectors'][idx]                         # Data to put in network
network    = forward_propagation(network, entry, transfert_mode) # Forward propagation 
net_output = network[-1]['O']                                    # Predicted value from network
predicted_values[i] = class_from_output(net_output)              # Storing the predicted class


print("entry          :", testing_set['classes'][idx])
plt.imshow(entry.reshape(28, 28), cmap='gray')
print("network output :", predicted_values[idx])